In [ ]:
%pip install streamlit
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.9 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import ast
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import requests
from io import StringIO
import os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


In [ ]:
# Cell 2: Load data from Google Drive
def load_data_from_drive():
    movies_path = '/content/drive/MyDrive/AI/movies_metadata.csv'
    credits_path = '/content/drive/MyDrive/AI/credits.csv'
    ratings_path = '/content/drive/MyDrive/AI/ratings_small.csv'

    movies = pd.read_csv(movies_path)
    credits = pd.read_csv(credits_path)
    ratings = pd.read_csv(ratings_path)

    # Filter movies with more votes for faster processing
    movies = movies[movies['vote_count'] > 500]

 # Convert IDs
    movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
    credits['id'] = pd.to_numeric(credits['id'], errors='coerce')
    ratings['movieId'] = pd.to_numeric(ratings['movieId'], errors='coerce')
    movies = movies.dropna(subset=['id'])
    credits = credits.dropna(subset=['id'])
    ratings = ratings.dropna(subset=['movieId'])
    movies['id'] = movies['id'].astype(int)
    credits['id'] = credits['id'].astype(int)
    ratings['movieId'] = ratings['movieId'].astype(int)

    # Merge movies + credits
    movies = movies.merge(credits, on='id', how='inner')

    # Clean features
    movies['overview'] = movies['overview'].fillna('')
    movies['tagline'] = movies['tagline'].fillna('')
    movies['description'] = movies['overview'] + " " + movies['tagline']

    movies = movies[['id', 'title', 'description', 'genres', 'cast', 'crew']]

    # Parsing helpers
    def parse_genres(obj):
        try: return [i['name'] for i in ast.literal_eval(obj)]
        except: return []
    def parse_cast(obj):
        try: return [i['name'] for i in ast.literal_eval(obj)[:3]]
        except: return []
    def parse_crew(obj):
        try: return [i['name'] for i in ast.literal_eval(obj) if i['job']=='Director']
        except: return []

    movies['genres'] = movies['genres'].apply(parse_genres)
    movies['cast'] = movies['cast'].apply(parse_cast)
    movies['crew'] = movies['crew'].apply(parse_crew)

    # Convert lists → strings
    movies['genres'] = movies['genres'].apply(lambda x: " ".join(x))
    movies['cast'] = movies['cast'].apply(lambda x: " ".join(x))
    movies['crew'] = movies['crew'].apply(lambda x: " ".join(x))

    movies['final_features'] = (
        movies['description'] + ' ' +
        movies['genres'] + ' ' +
        movies['cast'] + ' ' +
        movies['crew']
    )
    return movies, ratings

In [ ]:
# Streamlit page config
st.set_page_config(
    page_title="Movie Recommender System",
    page_icon="🎬",
    layout="wide",
    initial_sidebar_state="expanded"
)

MOVIES_URL = "https://drive.google.com/uc?export=download&id=1GOuUEu1-KgepbjTxIOkbAU8VNJ5lfEg3"
CREDITS_URL = "https://drive.google.com/uc?export=download&id=10iuK9C87fYLyDLJhqT3bpVv1A2IErmHR"
RATINGS_URL = "https://drive.google.com/uc?export=download&id=122XJoryYXvv3AUa6F_y1KiCcYdXQjEp4"

@st.cache_data
def load_data_from_url(url):
    response = requests.get(url)
    response.raise_for_status()
    return pd.read_csv(StringIO(response.text))

@st.cache_resource
def load_data():
    with st.spinner("Loading datasets..."):
        movies = load_data_from_url(MOVIES_URL)
        credits = load_data_from_url(CREDITS_URL)
        ratings = load_data_from_url(RATINGS_URL)

    # Filter movies with more votes for faster dev
    movies = movies[movies['vote_count'] > 500]

    # Convert IDs
    movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
    credits['id'] = pd.to_numeric(credits['id'], errors='coerce')
    ratings['movieId'] = pd.to_numeric(ratings['movieId'], errors='coerce')
    movies = movies.dropna(subset=['id'])
    credits = credits.dropna(subset=['id'])
    ratings = ratings.dropna(subset=['movieId'])
    movies['id'] = movies['id'].astype(int)
    credits['id'] = credits['id'].astype(int)
    ratings['movieId'] = ratings['movieId'].astype(int)

    # Merge movies + credits
    movies = movies.merge(credits, on='id', how='inner')

    # Clean features
    movies['overview'] = movies['overview'].fillna('')
    movies['tagline'] = movies['tagline'].fillna('')
    movies['description'] = movies['overview'] + " " + movies['tagline']

    movies = movies[['id', 'title', 'description', 'genres', 'cast', 'crew']]

    # Parsing helpers
    def parse_genres(obj):
        try: return [i['name'] for i in ast.literal_eval(obj)]
        except: return []
    def parse_cast(obj):
        try: return [i['name'] for i in ast.literal_eval(obj)[:3]]
        except: return []
    def parse_crew(obj):
        try: return [i['name'] for i in ast.literal_eval(obj) if i['job']=='Director']
        except: return []

    movies['genres'] = movies['genres'].apply(parse_genres)
    movies['cast'] = movies['cast'].apply(parse_cast)
    movies['crew'] = movies['crew'].apply(parse_crew)

    # Convert lists → strings
    movies['genres'] = movies['genres'].apply(lambda x: " ".join(x))
    movies['cast'] = movies['cast'].apply(lambda x: " ".join(x))
    movies['crew'] = movies['crew'].apply(lambda x: " ".join(x))

    movies['final_features'] = (
        movies['description'] + ' ' +
        movies['genres'] + ' ' +
        movies['cast'] + ' ' +
        movies['crew']
    )

    return movies, ratings

2025-09-07 15:25:15.043 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-07 15:25:15.049 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [ ]:
# Cell 3: Create TF-IDF model
@st.cache_resource
def create_tfidf_model(movies, pickle_path="/content/drive/MyDrive/tfidf_model.pkl"):
    # If pickle exists, load it
    try:
        with open(pickle_path, "rb") as f:
            tfidf, vectors = pickle.load(f)
        print("Loaded TF-IDF model from pickle")
    except:
        print("Creating new TF-IDF model...")
        tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
        vectors = tfidf.fit_transform(movies['final_features'])
        # Save to pickle
        with open(pickle_path, "wb") as f:
            pickle.dump((tfidf, vectors), f)
        print("TF-IDF model created and saved")
    return tfidf, vectors


In [ ]:
# Cell 4: Prepare collaborative data
@st.cache_resource
def prepare_collaborative_data(movies, ratings):
    movies_cf = movies[['id','title']].rename(columns={'id':'movieId'})
    ratings = ratings.merge(movies_cf, on="movieId", how="inner")

    user_mapping = {1: "Bob", 2: "Alice", 3: "Charlie", 4: "Diana", 5: "Eve"}
    ratings['user_name'] = ratings['userId'].replace(user_mapping)

    user_item_matrix = ratings.pivot_table(index='user_name', columns='title', values='rating').fillna(0)

    from sklearn.metrics.pairwise import cosine_similarity
    user_sim = cosine_similarity(user_item_matrix)
    user_sim_df = pd.DataFrame(user_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

    return ratings, user_item_matrix, user_sim_df


In [ ]:
# Cell 5: Recommendation functions
# Content-based
def content_based_recommend(movie_title, movies, vectors, top_n=10):
    if movie_title not in movies['title'].values:
        return "❌ Movie not found", []
    idx = movies[movies['title']==movie_title].index[0]
    scores = linear_kernel(vectors[idx], vectors).flatten()
    indices = scores.argsort()[-(top_n+1):-1][::-1]
    return "Content-based recommendations", [(movies.iloc[i].title, float(scores[i])) for i in indices]

# Collaborative
def collaborative_recommend(user_name, user_item_matrix, user_sim_df, top_n=50):
    if user_name not in user_item_matrix.index:
        return {}
    sim_scores = user_sim_df[user_name].drop(user_name).sort_values(ascending=False)
    top_users = sim_scores.index[:5]
    neighbor_ratings = user_item_matrix.loc[top_users].mean(axis=0)
    watched = user_item_matrix.loc[user_name][user_item_matrix.loc[user_name]>0].index
    neighbor_ratings = neighbor_ratings.drop(watched, errors='ignore')
    top_recs = neighbor_ratings.sort_values(ascending=False).head(top_n)
    return {title: score for title, score in top_recs.items()}

# Hybrid
def hybrid_recommend(user_name, liked_movie, movies, vectors, user_item_matrix, user_sim_df, alpha=0.5, top_n=10):
    if not user_name or user_name.strip()=="" or user_name=="-":
        user_name = f"User_{random.randint(6,671)}"
    collab_scores = collaborative_recommend(user_name, user_item_matrix, user_sim_df, top_n=50)
    if liked_movie not in movies['title'].values:
        return user_name, [("❌ Movie not found", 0.0)]
    idx = movies.index[movies['title']==liked_movie][0]
    cs = linear_kernel(vectors[idx], vectors).flatten()
    content_scores = {movies.iloc[i].title: float(cs[i]) for i in cs.argsort()[-51:-1]}
    all_titles = set(collab_scores.keys()) | set(content_scores.keys())
    hybrid_scores = {t: alpha*content_scores.get(t,0)+ (1-alpha)*collab_scores.get(t,0) for t in all_titles}
    ranked = sorted(hybrid_scores.items(), key=lambda x:x[1], reverse=True)[:top_n]
    return user_name, [(t,float(s)) for t,s in ranked]


In [ ]:
# ----- Evaluation Functions -----
def content_predict(user_id, movie_id, movies, ratings_aggregated, content_similarity):
    if movie_id not in movies['id'].values:
        return None
    idx = movies[movies['id'] == movie_id].index[0]
    sims = content_similarity[idx]
    user_ratings = ratings_aggregated[ratings_aggregated['userId'] == user_id]
    if user_ratings.empty:
        return None
    sim_scores = []
    for _, row in user_ratings.iterrows():
        if row['movieId'] in movies['id'].values:
            jdx = movies[movies['id'] == row['movieId']].index[0]
            sim_scores.append((row['rating'], sims[jdx]))
    if not sim_scores:
        return None
    weighted_sum = sum(r*s for r,s in sim_scores)
    sim_sum = sum(s for _,s in sim_scores)
    return weighted_sum/sim_sum if sim_sum != 0 else None

# Aggregate duplicate ratings by taking the mean before pivoting
# This is now done outside the function

def collab_predict(user_id, movie_id, ratings_matrix, user_sim_df):
    if movie_id not in ratings_matrix.columns:
        return None
    if user_id not in user_sim_df.index:
        return None
    sims = user_sim_df[user_id].drop(user_id, errors='ignore')
    top_users = sims.sort_values(ascending=False).head(5).index

    # Check if top_users is empty
    if top_users.empty:
        return None

    # Ensure movie_id is in the columns of ratings_matrix for top_users
    if movie_id not in ratings_matrix.columns:
        return None

    top_ratings = ratings_matrix.loc[top_users, movie_id]
    weights = sims.loc[top_users]

    # Handle cases where weights might be all zeros
    if weights.sum() == 0:
        return None

    return np.dot(top_ratings, weights)/weights.sum()

def hybrid_predict(user_id, movie_id, movies, ratings_aggregated, content_similarity, ratings_matrix, user_sim_df, alpha=0.5):
    cp = content_predict(user_id, movie_id, movies, ratings_aggregated, content_similarity)
    cf = collab_predict(user_id, movie_id, ratings_matrix, user_sim_df)
    if cp is None and cf is None:
        return None
    if cp is None: return cf
    if cf is None: return cp
    return alpha*cp + (1-alpha)*cf

def evaluate_model(predict_func, movies, ratings_aggregated, content_similarity, ratings_matrix, user_sim_df, n_samples=300):
    # Ensure the sampled ratings are in the aggregated dataframe
    test = ratings_aggregated.sample(n_samples, random_state=42)
    preds, truths = [], []
    for _, row in test.iterrows():
        # Pass necessary arguments to predict_func
        if predict_func.__name__ == 'content_predict':
            pred = predict_func(row['userId'], row['movieId'], movies, ratings_aggregated, content_similarity)
        elif predict_func.__name__ == 'collab_predict':
             pred = predict_func(row['userId'], row['movieId'], ratings_matrix, user_sim_df)
        elif predict_func.__name__ == 'hybrid_predict':
             pred = predict_func(row['userId'], row['movieId'], movies, ratings_aggregated, content_similarity, ratings_matrix, user_sim_df)
        else:
            pred = predict_func(row['userId'], row['movieId']) # For other predict functions

        if pred is not None:
            preds.append(pred)
            truths.append(row['rating'])
    if not preds:
        return None, None
    mse = mean_squared_error(truths, preds)
    rmse = np.sqrt(mse)
    return mse, rmse

# ----- Run Evaluation -----
# This part will be moved to the next cell (Cell 7) to ensure data loading and TF-IDF creation are done first.

In [ ]:
# Cell 7: Run evaluation and Demo recommendations

# Load data and create TF-IDF model
movies, ratings = load_data_from_drive()
tfidf, vectors = create_tfidf_model(movies)

# Prepare collaborative data
ratings_aggregated = ratings.groupby(['userId', 'movieId'])['rating'].mean().reset_index()
ratings_matrix = ratings_aggregated.pivot(index="userId", columns="movieId", values="rating").fillna(0)
user_sim = cosine_similarity(ratings_matrix)
user_sim_df_eval = pd.DataFrame(user_sim, index=ratings_matrix.index, columns=ratings_matrix.index) # Renamed to avoid conflict with user_sim_df in hybrid_recommend
content_similarity = linear_kernel(vectors, vectors) # Calculate content similarity matrix

print("Running evaluation...")
mse_content, rmse_content = evaluate_model(content_predict, movies, ratings_aggregated, content_similarity, ratings_matrix, user_sim_df_eval)
mse_collab, rmse_collab = evaluate_model(collab_predict, movies, ratings_aggregated, content_similarity, ratings_matrix, user_sim_df_eval) # Removed movies, content_similarity as they are not used in collab_predict within evaluate_model
mse_hybrid, rmse_hybrid = evaluate_model(hybrid_predict, movies, ratings_aggregated, content_similarity, ratings_matrix, user_sim_df_eval)


results_df = pd.DataFrame({
    "System": ["Content-Based", "Collaborative", "Hybrid"],
    "MSE": [mse_content, mse_collab, mse_hybrid],
    "RMSE": [rmse_content, rmse_collab, rmse_hybrid]
})

print("📊 Evaluation Results:")
display(results_df)

# Cell 8: Visualization
plt.figure(figsize=(8,4))
sns.barplot(x="System", y="RMSE", data=results_df, palette="viridis")
plt.title("RMSE Comparison (Lower is Better)")
plt.show()

plt.figure(figsize=(8,4))
sns.barplot(x="System", y="MSE", data=results_df, palette="magma")
plt.title("MSE Comparison (Lower is Better)")
plt.show()

# Cell 9: Demo recommendations
def demo_recommendations(movies, vectors, user_item_matrix, user_sim_df):
    print("=== MOVIE RECOMMENDER SYSTEM DEMO ===")

    # Content-based recommendation demo
    print("\n1. Content-Based Recommendations:")
    movie_title = "The Dark Knight"  # Example movie
    if movie_title in movies['title'].values:
        method_name, recs = content_based_recommend(movie_title, movies, vectors, top_n=5)
        print(f"Top 5 content-based recommendations for '{movie_title}':")
        for i, (title, score) in enumerate(recs, 1):
            print(f"{i}. {title} (score: {score:.4f})")
    else:
        print(f"Movie '{movie_title}' not found in database")

    # Hybrid recommendation demo
    print("\n2. Hybrid Recommendations:")
    user_name = "Bob"
    movie_title = "The Dark Knight"
    if movie_title in movies['title'].values and user_name in user_item_matrix.index:
        user_display, recs = hybrid_recommend(user_name, movie_title, movies, vectors, user_item_matrix, user_sim_df, top_n=5)
        print(f"Top 5 hybrid recommendations for user '{user_display}' based on '{movie_title}':")
        for i, (title, score) in enumerate(recs, 1):
            print(f"{i}. {title} (score: {score:.4f})")
    else:
        print(f"Movie '{movie_title}' or user '{user_name}' not found")

# Prepare data for demo recommendations
ratings_demo, user_item_matrix_demo, user_sim_df_demo = prepare_collaborative_data(movies, ratings) # Use original ratings for demo

# Run the demo
demo_recommendations(movies, vectors, user_item_matrix_demo, user_sim_df_demo)

/tmp/ipython-input-2957222789.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(movies_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AI/credits.csv'

In [ ]:

def main():
    st.title("🎬 Movie Recommender System")

    # Load data and models
    movies, ratings = load_data()
    tfidf, vectors = create_tfidf_model(movies)
    ratings, user_item_matrix, user_sim_df = prepare_collaborative_data(movies, ratings)

    st.header("Movie Recommendations")

    rec_type = st.radio("Recommendation Type", ["Content-Based", "Hybrid"])
    user_choices = ["-", "Bob", "Alice", "Charlie", "Diana", "Eve"]
    user_choice = st.selectbox("Select Movie Critic", user_choices) if rec_type=="Hybrid" else "-"

    movie_choices = sorted(movies['title'].unique())
    movie_title = st.selectbox("Select a Movie", movie_choices)
    num_recs = st.slider("Number of Recommendations", 1, 20, 10)

    if st.button("Get Recommendations"):
        if not movie_title.strip():
            st.error("❌ Please select a movie first.")
            return
        if rec_type=="Content-Based":
            method_name, recs = content_based_recommend(movie_title, movies, vectors, num_recs)
            user_display = "Content-Based Filtering"
        else:
            user_input = "" if user_choice=="-" else user_choice
            user_display, recs = hybrid_recommend(user_input, movie_title, movies, vectors, user_item_matrix, user_sim_df, top_n=num_recs)
            method_name = "Hybrid recommendations"

        if isinstance(recs,str) or len(recs)==0:
            st.error(recs if isinstance(recs,str) else "❌ No recommendations found")
        else:
            max_score = max([s for _,s in recs]) if recs else 1.0
            rows = [[f"{i}. {title}", f"{(score/max_score)*100:.1f}%"] for i,(title,score) in enumerate(recs,start=1)]
            st.subheader(f"🎭 {method_name} for {user_display} (based on {movie_title}):")
            st.table(pd.DataFrame(rows, columns=["Movie","Score"]))

if __name__=="__main__":
    main()

2025-09-07 15:20:15.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-07 15:20:15.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-07 15:20:15.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-07 15:20:15.688 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-07 15:20:15.689 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-07 15:20:15.690 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-07 15:20:15.692 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-07 15:20:15.707 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Loaded TF-IDF model from pickle
